In [8]:
import animius as am

In [5]:
# change working directory
os.chdir("/Users/xasiimov/Documents/Develop/Python/animius/animius")

In [3]:
# train SpeakerVerification model
parser = am.ParseSubtitle.Parser()

for i in range(1, 14):
    parser.load('resources/Violet_Evergarden/subtitles/' + str(i) + '.ass')
    parser.slice_audio('resources/Violet_Evergarden/audios/' + str(i) + '.mp3',
                       'resources/Violet_Evergarden/slices/' + str(i))

sv_model_config = am.ModelConfig(cls='SpeakerVerification')

sv_data = am.SpeakerVerificationData()
sv_data.add_text_file('resources/Violet_Evergarden/true.txt', is_speaker=True)
sv_data.add_text_file('resources/Violet_Evergarden/false.txt', is_speaker=False)

sv_model = am.SpeakerVerification.SpeakerVerificationModel()
sv_model.build_graph(sv_model_config, sv_data)

sv_model.init_tensorflow()
sv_model.train()
sv_model.save(r'resources/models')

FileNotFoundError: [Errno 2] No such file or directory: 'resources/Violet_Evergarden/subtitles/1.ass'

In [9]:
# use SpeakerVerification to predict audio slices
# train Chatbot Model

result = []

chatbot_data = am.ChatData()
glove_embedding = am.WordEmbedding()
glove_embedding.create_embedding('/Users/xasiimov/Documents/Develop/Python/Dataset/GloVe/glove.twitter.27B.50d.txt',
                                 vocab_size=40000)
chatbot_data.add_embedding_class(glove_embedding)

for i in range(1, 14):
    sv_data = am.SpeakerVerificationData()
    sv_data.add_text_file('resources/Violet_Evergarden/true.txt', is_speaker=True)
    sv_data.add_text_file('resources/Violet_Evergarden/false.txt', is_speaker=False)

    sv_model = am.Model.load(r'resources/models/sv_model_1', data=sv_data)

    sv_data.add_folder('resources/Violet_Evergarden/slices/' + str(i) + '/', is_speaker=None)
    result = sv_model.predict(sv_data, save_path='resources/Violet_Evergarden/results/' + str(i) + '_result.txt')

    parser = am.ParseSubtitle.Parser()
    parser.load('resources/Violet_Evergarden/subtitles/' + str(i) + '.ass')
    parser.parse_audio_sentences()

    convs = parser.detect_conversation(result)
    chatbot_data.add_data(convs)

chatbot_data.add_twitter("/Users/xasiimov/Documents/Develop/Python/Dataset/twitter_en/twitter_en.txt", 0, 7500)
chatbot_data.save(r'resources/data/Chatbot Data/', name='ChatbotData')

AttributeError: module 'animius' has no attribute 'ChatData'

In [11]:
# train IntentNER model

intentNER_mc = am.ModelConfig(cls='IntentNER')
intentNER_data = am.IntentNERData()
intentNER_data = am.IntentNERData()
intentNER_model = am.IntentNER.IntentNERModel()

glove_embedding = am.WordEmbedding()
glove_embedding.create_embedding('/Users/xasiimov/Documents/Develop/Python/Dataset/GloVe/glove.twitter.27B.50d.txt',
                                 vocab_size=40000)
intentNER_data.add_embedding_class(glove_embedding)

intentNER_data.set_intent_folder('resources/data/IntentNER Data')

intentNER_data.save(r'resources/data/IntentNER Data/', name='IntentNERData')

intentNER_model.build_graph(intentNER_mc, intentNER_data)
intentNER_model.init_tensorflow()

intentNER_model.train()

intentNER_model.save(directory='resources/models/intent_ner', name='Intent NER')

intentNER_model = am.Model.load(directory='resources/models/intent_ner', name='Intent NER')
intentNER_mc = intentNER_model.model_config()

glove_embedding = am.WordEmbedding()
glove_embedding.create_embedding('/Users/xasiimov/Documents/Develop/Python/Dataset/GloVe/glove.twitter.27B.50d.txt',
                                 vocab_size=40000)

input_data = intentNER_model.data
input_data.add_embedding_class(glove_embedding)
input_data.set_input('how is the weather in san fransisco?')
print(intentNER_model.predict(input_data))

TypeError: 'module' object is not callable